In [ ]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2021-08-31 10:26:55--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2021-08-31 10:26:55--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc41307ad30891694f69e5a978e3.dl.dropboxusercontent.com/cd/0/inline/BVQSR5DmzMzrBe2k3b6QV9eC-CazorPJShG9ZSCLZPvuvjv6TPORawd4X3YdfIt_UjGDno3Iqs4rPsNR8NtAv2p-7kFNIOTDKudGaLU6V5g82AteRjKYzJLDAVPGgpWJUpknYbyvxOiUb7zYlWlKlXd2/file# [following]
--2021-08-31 10:26:56--  https://uc41307ad30891694f69e5a978e3.dl.dropboxusercontent.com/cd/0/inline/BVQSR5DmzMzrBe2k3b6QV9eC-CazorPJShG9ZSCLZPvuvjv6T

In [ ]:
# Let's unzip the file
!unzip -q "/content/movie-reviews-dataset.zip"

In [ ]:
from tensorflow.keras.preprocessing import  text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, RNN, SimpleRNNCell, Embedding, Dropout

In [ ]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
      lambda text, label: (regex_replace(text, "<br />", " "), label),
  )

In [ ]:
train_data = prepareData("/content/movie-reviews-dataset/train")
test_data = prepareData("/content/movie-reviews-dataset/test")

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b'A CRY IN THE DARK   A CRY IN THE DARK was a film that I anticipated would offer a phenomenal performance from Meryl Streep and a solid, if unremarkable film. This assumption came from the fact that aside from Streep\'s Best Actress nomination, the movie received little attention from major awards groups.  Little did I anticipate that A CRY IN THE DARK would be such a riveting drama, well-constructed on every level. If you ask me, this is an under-appreciatted classic.  The film opens rather slowly, letting the audience settle into the Chamberlain\'s at a relaxed pace and really notice that, at the core, they are an incredibly loving, simple family. Fred Schepisi (the director) selects random moments to capture of a family on vacation that give a looming sense of the oncoming tragedy, while also showing the attentive bliss with which Lindy (Streep) and Michael (Sam Neill) Chamberlain care for their chi

In [ ]:
model =Sequential()

In [ ]:
model.add(Input(shape=(1,), dtype="string"))

In [ ]:
max_tokens= 1000
max_len = 100
vectorize_layer = TextVectorization(
    max_tokens = max_tokens,
    output_mode = 'int',
    output_sequence_length = max_len,
)

In [ ]:
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

model.add(vectorize_layer)

In [ ]:
model.add(Embedding(max_tokens + 1, 128))

rnn = RNN(SimpleRNNCell(64), return_sequences=False, return_state=False)
model.add(rnn)
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 32s 39ms/step - loss: 0.6974 - accuracy: 0.5080
Epoch 2/10
782/782 [==============================] - 30s 39ms/step - loss: 0.6912 - accuracy: 0.5325
Epoch 3/10
782/782 [==============================] - 30s 39ms/step - loss: 0.6795 - accuracy: 0.5650
Epoch 4/10
782/782 [==============================] - 30s 38ms/step - loss: 0.6760 - accuracy: 0.5767
Epoch 5/10
782/782 [==============================] - 30s 39ms/step - loss: 0.6774 - accuracy: 0.5676
Epoch 6/10
782/782 [==============================] - 31s 39ms/step - loss: 0.6552 - accuracy: 0.6108
Epoch 7/10
782/782 [==============================] - 30s 39ms/step - loss: 0.6470 - accuracy: 0.6253
Epoch 8/10
782/782 [==============================] - 30s 38ms/step - loss: 0.6319 - accuracy: 0.6477
Epoch 9/10
782/782 [==============================] - 30s 38ms/step - loss: 0.6378 - accuracy: 0.6390
Epoch 10/10
782/782 [==============================] - 30s 38ms/step - loss: 0.609

In [ ]:
model.evaluate(test_data)

782/782 [==============================] - 13s 16ms/step - loss: 0.6366 - accuracy: 0.6529


[0.6365990042686462, 0.6529200077056885]

In [ ]:
text = "This movis is bad"

In [ ]:
model.predict([text]) # 0 = Negative review, 1 = positive review

array([[0.4494273]], dtype=float32)

In [ ]:
text2 = "This movis is good"

In [ ]:
model.predict([text2]) # 0 = Negative review, 1 = positive review

array([[0.74964345]], dtype=float32)